In [1]:
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

#import os
#pathName = '/home/ubuntu/data/PEC/model_dict/ml100k/'
#if not os.path.exists(pathName):
#     os.makedirs(pathName)
#MODELPATH = pathName + 'dnn_v1.0_'

data_path = '/home/zhangxz/workspace/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv')
simUsers = np.loadtxt(data_path+"simUser.csv",  delimiter=',',dtype=int)

trainUIT = UIT[UIT['day']<max(UIT['day']+1)*0.6]
validUIT = UIT[UIT['day']<max(UIT['day']+1)*0.8]

contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())

rewardPara = {"alpha":0.01,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 20

BATCH_SIZE = 128
GAMMA = 0.99

num_episodes = 10
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = trainUIT.shape[0]*3
agentStep = 0

torch.cuda.empty_cache()
UIT.shape,trainUIT.shape,validUIT.shape

((300983, 12), (198175, 12), (253630, 12))

In [2]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE_random(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
       # self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1

            actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
        
            if self.W[-1] not in actionIndex:
                actionIndex.pop()
            for index in actionIndex:
                action[index] = 1           
            self.lastAction = action
            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action


In [3]:
bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE_random(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 17 00:16:16 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.66793  0.00378  0.06152] total reward: -0.60263
UEHitrate: 0.0053  edgeHitrate 0.0769 sumHitrate 0.0822  privacy: 3.98917

--Time: Sun Oct 17 00:16:42 2021   Index: 19999   Loss: 0.0 --
Reward: [-0.55522  0.00441  0.0616 ] total reward: -0.48921
UEHitrate: 0.00585  edgeHitrate 0.077 sumHitrate 0.08285  privacy: 3.35539

--Time: Sun Oct 17 00:17:21 2021   Index: 29999   Loss: 0.0 --
Reward: [-0.48657  0.00482  0.0596 ] total reward: -0.42215
UEHitrate: 0.0061  edgeHitrate 0.0745 sumHitrate 0.0806  privacy: 2.94

--Time: Sun Oct 17 00:18:06 2021   Index: 39999   Loss: 0.0 --
Reward: [-0.43571  0.00474  0.06008] total reward: -0.37089
UEHitrate: 0.00608  edgeHitrate 0.0751 sumHitrate 0.08118  privacy: 2.64643

--Time: Sun Oct 17 00:18:49 2021   Index: 49999   Loss: 0.0 --
Reward: [-0.39415  0.00502  0.05973] total reward: -0.32941
UEHitrate: 0.00644  edgeHitrate 0.07466 sumHitrate 0.0811  privacy: 2.40865

--Time: S

In [4]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.0822 , 0.08285, 0.0806 , 0.08118, 0.0811 , 0.08047, 0.07807,
        0.07765, 0.07756, 0.07671, 0.07609, 0.07615, 0.07572, 0.07578,
        0.07563, 0.07551, 0.07535, 0.07569, 0.07552, 0.07561, 0.076  ,
        0.07587, 0.07563, 0.0755 , 0.07586, 0.07587, 0.07579, 0.07606,
        0.07628, 0.07639, 0.0764 ]),
 array([0.0053 , 0.00585, 0.0061 , 0.00608, 0.00644, 0.00642, 0.00621,
        0.00618, 0.00628, 0.0062 , 0.00627, 0.00631, 0.00652, 0.00667,
        0.00671, 0.00689, 0.00695, 0.00706, 0.00708, 0.00712, 0.0072 ,
        0.00726, 0.00726, 0.00727, 0.00732, 0.00735, 0.00744, 0.0075 ,
        0.00751, 0.0075 , 0.00749]),
 array([0.0769 , 0.077  , 0.0745 , 0.0751 , 0.07466, 0.07405, 0.07186,
        0.07147, 0.07128, 0.07051, 0.06982, 0.06984, 0.06919, 0.06911,
        0.06892, 0.06863, 0.0684 , 0.06864, 0.06844, 0.06849, 0.0688 ,
        0.0686 , 0.06837, 0.06822, 0.06854, 0.06852, 0.06834, 0.06856,
        0.06877, 0.06889, 0.06891]))

In [5]:
privacyReduction

array([3.98917, 3.35539, 2.94   , 2.64643, 2.40865, 2.22707, 2.07864,
       1.94161, 1.81692, 1.69633, 1.58983, 1.49978, 1.41597, 1.33826,
       1.26797, 1.20028, 1.1399 , 1.08002, 1.02384, 0.97439, 0.92599,
       0.88191, 0.83733, 0.7954 , 0.75647, 0.72131, 0.68817, 0.6563 ,
       0.62479, 0.597  , 0.59438])

In [6]:
class UE_None(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
       # self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1      
            self.lastAction = action
            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action


In [7]:
bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE_None(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 17 00:35:57 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.04246  0.00184  0.30576] total reward: 0.26514
UEHitrate: 0.0028  edgeHitrate 0.3822 sumHitrate 0.385  privacy: 1.0

--Time: Sun Oct 17 00:36:20 2021   Index: 19999   Loss: 0.0 --
Reward: [-0.03706  0.00252  0.33908] total reward: 0.30454
UEHitrate: 0.0036  edgeHitrate 0.42385 sumHitrate 0.42745  privacy: 1.0

--Time: Sun Oct 17 00:36:44 2021   Index: 29999   Loss: 0.0 --
Reward: [-0.03487  0.00289  0.32576] total reward: 0.29378
UEHitrate: 0.00403  edgeHitrate 0.4072 sumHitrate 0.41123  privacy: 1.0

--Time: Sun Oct 17 00:37:09 2021   Index: 39999   Loss: 0.0 --
Reward: [-0.03305  0.00298  0.31552] total reward: 0.28545
UEHitrate: 0.00407  edgeHitrate 0.3944 sumHitrate 0.39848  privacy: 1.0

--Time: Sun Oct 17 00:37:30 2021   Index: 49999   Loss: 0.0 --
Reward: [-0.03114  0.0032   0.3135 ] total reward: 0.28556
UEHitrate: 0.0044  edgeHitrate 0.39188 sumHitrate 0.39628  privacy: 1.0

--Time: Sun Oct 17 00:37:57 2

In [8]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.385  , 0.42745, 0.41123, 0.39848, 0.39628, 0.39353, 0.38591,
        0.38965, 0.38726, 0.38685, 0.38734, 0.38693, 0.38312, 0.38718,
        0.39145, 0.38885, 0.39204, 0.39449, 0.39623, 0.39668, 0.39839,
        0.39664, 0.39534, 0.39411, 0.39457, 0.39255, 0.39052, 0.39051,
        0.39281, 0.39617, 0.39647]),
 array([0.0028 , 0.0036 , 0.00403, 0.00407, 0.0044 , 0.00443, 0.00424,
        0.00426, 0.00432, 0.00428, 0.00443, 0.00449, 0.0047 , 0.00483,
        0.00489, 0.00507, 0.00511, 0.0052 , 0.00523, 0.00526, 0.00536,
        0.00542, 0.00545, 0.00549, 0.00552, 0.00553, 0.0056 , 0.00564,
        0.00565, 0.00564, 0.00563]),
 array([0.3822 , 0.42385, 0.4072 , 0.3944 , 0.39188, 0.3891 , 0.38167,
        0.38539, 0.38293, 0.38257, 0.38291, 0.38244, 0.37842, 0.38235,
        0.38656, 0.38378, 0.38692, 0.38929, 0.391  , 0.39142, 0.39303,
        0.39122, 0.38989, 0.38862, 0.38904, 0.38702, 0.38492, 0.38488,
        0.38716, 0.39053, 0.39084]))

In [9]:
privacyReduction

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])